### Spiegel - Klimawandel

In [1]:
import requests
from bs4 import BeautifulSoup
import html5lib
import numpy as np
import json

In [2]:
# define page url -- necessary for functions below
url = 'https://www.spiegel.de/thema/klimawandel/'
main = 'https://www.spiegel.de/'
archive = ['https://www.spiegel.de/thema/klimawandel/dossierarchiv-', '.html']
# add a number for complete page, e.g. https://www.spiegel.de/thema/klimawandel/dossierarchiv-2.html
# it works if we dont't want automatically scraping all the articles but only some archive pages, e.g. 1 - 100.

In [16]:
def create_archive_link(n):
    return ''.join([archive[0], str(n), archive[1]])

In [4]:
# get page
def get_page(url):
    return BeautifulSoup(requests.get(url).content, 'html.parser')

In [5]:
# find article links and add main url
def find_and_complete_links(page):
    article_links = [i.find('a')['href'] for i in page.find_all('h2', class_='article-title')]
    article_links = list([main + i for i in article_links if not i.startswith(main)] + [i for i in article_links if i.startswith(main)])
    return article_links

In [6]:
#get and return tuple (headline, headline intro, article intro)
def get_headline_intro(art):
    art_title = art.find_all('h2', class_='article-title lp-article-title')
    if len(art_title) == 0:
        headline, headline_intro = np.nan, np.nan
    try:
        for i in art_title:
            try:
                headline = i.find('span', class_='headline').get_text()
            except:
                headline = np.nan
            try:
                headline_intro = i.find('span', class_='headline-intro').get_text()
            except:
                headline_intro = np.nan
    except:
        headline, headline_intro = np.nan, np.nan
    try:
        intro = art.find_all('p', class_='article-intro')[0].get_text()
    except:
        intro = np.nan
    return (headline, headline_intro, intro)

In [7]:
#get article text and remove tags
def get_clean_text(art):
    return ' '.join([i.get_text().strip('<b>\n') for i in art.find_all('div', class_='article-section clearfix')[0].findAll('p')])

In [8]:
# get date and time as list [date, time]
def get_date_time(art):
    return [i.strip('\r\n\t') for i in art.find('span', class_ = "article-function-date").find('time').get_text().split('\xa0')]

In [9]:
# get forum link
def get_forum_link(art):
    forum_link = art.find_all("div", class_="module-subtitle forum-title")[0].find('a')['href']
    if not forum_link.startswith(main):
        forum_link = main + forum_link
    return forum_link

In [10]:
def clean_str(s, chars='\n\t\r', strip_chars='\.', only_lstrip=False):
    for char in chars:
        s = s.replace(char, '')
    if only_lstrip:
        for char in strip_chars:
            s = s.lstrip(char)
    else:
        for char in strip_chars:
            s = s.strip(char)
    return s

In [11]:
def get_comments(forum_link, BREAK_NUM = 5000, title_sep=' '):
    all_comments = []
    i = 0
    thread = BeautifulSoup(requests.get(forum_link).content, 'html.parser')
    while True:
        comment_info = thread.find_all('div', 'postbit clearfix')
        for n, comment in enumerate(comment_info):
            all_comments.append([clean_str(comment.find('div', class_='article-comment-title').get_text()) + title_sep +\
                                clean_str(comment.find('span', class_="postContent").get_text(), only_lstrip=True)])
        page_count = thread.find('span', class_='page-count').get_text().split()[1::2]
        if page_count[0] == page_count[1] or i == BREAK_NUM: #last page or BREAK_NUM 
            break
        link_next = main + thread.find('a', class_='page-next')['href']
        thread = BeautifulSoup(requests.get(link_next).content, 'html.parser')
        i += 1
    return all_comments

In [12]:
def write_article(article_dict, filename):
    with open(filename, 'w') as outfile:
        json.dump(article_dict, outfile)

In [150]:
# it's an example how you can handle json files
import json
# write dict as json file
with open('article1.txt', 'w') as outfile:
    json.dump(article_dict, outfile)
# read json file
with open('article1.txt') as json_file:
    data = json.load(json_file)

In [14]:
def scrape_page(url):
    page = get_page(url)
    article_links = find_and_complete_links(page)
    for link in article_links:
        print(link)
        try:
            art = get_page(link)
        except:
            print('smth is wrong with get_page')
            return 0
        try:
            headline, headline_intro, intro = get_headline_intro(art)
        except:
            headline, headline_intro, intro = None, None, None
        try:
            date_time = get_date_time(art)
            date = date_time[0]
            time = date_time[1]
        except:
            date, time = None, None
        try:
            text = get_clean_text(art)
        except:
            text = None
        try:
            comments = get_comments(get_forum_link(art))
        except:
            comments = None
        
        article_dict = {'headline': headline,
                        'headline_intro': headline_intro, 
                        'date': date,
                        'time': time, 
                        'intro': intro, 
                        'text': text,
                        'thread': comments}
        
        yield article_dict

In [55]:
MAX_NUM = 100 # the latest page num
FIRST_NUM = 50 #archive starts with page ...

# with first page
#page_urls = [url] + list(map(create_archive_link, range(FIRST_NUM, MAX_NUM)))
#urls_and_pnum = zip(page_urls, list([1] + list(range(FIRST_NUM, MAX_NUM))))

#without first page
page_urls = list(map(create_archive_link, range(FIRST_NUM, MAX_NUM)))
urls_and_pnum = zip(page_urls, list(range(FIRST_NUM, MAX_NUM)))

In [56]:
for link, page_num in urls_and_pnum:
    print(page_num)
    for art_num, art in enumerate(scrape_page(link)):
        write_article(art, 'C:\\Users\\Даша\\Downloads\\FH\\DQDA\\Projekt\\corpus\\article_' + str(page_num) + 'p' + str(art_num) + '.txt')

50
https://www.spiegel.de//spiegel/print/d-159073568.html
https://www.spiegel.de//spiegel/print/d-159073529.html
https://www.spiegel.de//spiegel/print/d-158955180.html
https://www.spiegel.de//spiegel/print/d-158955142.html
https://www.spiegel.de//spiegel/print/d-158957323.html
https://www.spiegel.de/politik/deutschland/nachrichten-am-morgen-die-news-in-echtzeit-a-1225723.html
https://www.spiegel.de/wissenschaft/natur/klimawandel-wenn-viel-reden-nur-wenig-hilft-a-1225586.html
https://www.spiegel.de/wissenschaft/natur/neandertaler-extremkaelte-brachte-den-tod-a-1225268.html
https://www.spiegel.de/wissenschaft/natur/eisloch-vor-groenland-was-ist-da-los-a-1224643.html
https://www.spiegel.de/wissenschaft/mensch/klimaziele-2020-so-koennte-es-fuer-deutschland-noch-klappen-a-1223443.html
51
https://www.spiegel.de//spiegel/print/d-158846341.html
https://www.spiegel.de/wissenschaft/natur/klima-alarmismus-ist-nicht-unser-hauptproblem-kommentar-a-1223315.html
https://www.spiegel.de/wissenschaft/me

https://www.spiegel.de/wissenschaft/natur/klimaziele-2020-realismus-und-beharren-auf-alten-gewohnheiten-kommentar-a-1186964.html
https://www.spiegel.de/wissenschaft/natur/great-barrier-reef-erderwaermung-fuehrt-zu-mehr-weiblichen-meeresschildkroeten-a-1186985.html
https://www.spiegel.de/wirtschaft/soziales/klimapolitik-merkels-versagen-in-einem-chart-a-1186948.html
https://www.spiegel.de/wirtschaft/soziales/grosse-koalition-zu-klimaschutz-union-und-spd-wollen-ziel-2020-aufgeben-a-1186785.html
https://www.spiegel.de/wissenschaft/natur/mehr-fluechtlinge-durch-klimawandel-asyl-studie-entsetzt-wissenschaftler-a-1184640.html
https://www.spiegel.de/wissenschaft/natur/paris-staaten-entdecken-geldquellen-gegen-klimawandel-a-1183001.html
https://www.spiegel.de/wissenschaft/natur/usa-koennen-klimaziele-trotz-donald-trump-erreichen-sagen-kerry-und-bloomerg-a-1182934.html
https://www.spiegel.de/wissenschaft/natur/emmanuel-macron-als-klimabettler-beim-klimagipfel-in-paris-a-1182866.html
59
https://

https://www.spiegel.de/wissenschaft/natur/erderwaermung-beeinflusst-farbe-grosser-seen-a-1169683.html
https://www.spiegel.de/wissenschaft/natur/usa-rex-tillerson-haelt-verbleib-im-pariser-klimavertrag-fuer-moeglich-a-1168363.html
66
https://www.spiegel.de//spiegel/print/d-153158014.html
https://www.spiegel.de//spiegel/print/d-152593480.html
https://www.spiegel.de/wissenschaft/natur/klimawandel-nordsee-1-67-grad-waermer-als-1965-a-1166932.html
https://www.spiegel.de/wissenschaft/natur/antarktis-vulkan-leitete-vor-18-000-jahren-klimawandel-ein-a-1166153.html
https://www.spiegel.de/wissenschaft/mensch/stromverbrauch-in-suedeuropa-koennte-durch-klimawandel-steigen-a-1164980.html
https://www.spiegel.de/wissenschaft/natur/studie-exxonmobil-soll-oeffentlichkeit-beim-klimawandel-irregefuehrt-haben-a-1164218.html
https://www.spiegel.de/wissenschaft/mensch/camp-century-in-groenland-verlassene-us-basis-untersucht-a-1163456.html
https://www.spiegel.de/wissenschaft/natur/trump-forrest-zehntausende-

https://www.spiegel.de/wissenschaft/natur/donald-trump-was-die-welt-gegen-den-klimakiller-tun-kann-a-1149886.html
https://www.spiegel.de/wissenschaft/natur/donald-trump-wie-die-usa-aus-dem-pariser-klimavertrag-aussteigen-koennten-a-1149713.html
https://www.spiegel.de/politik/ausland/g7-gipfel-auf-sizilien-merkel-wirft-usa-blockade-bei-klimapolitik-vor-a-1149512.html
https://www.spiegel.de/politik/ausland/donald-trump-verweigert-sich-bei-g7-gipfel-der-klimaschutz-erklaerung-a-1149498.html
https://www.spiegel.de/wissenschaft/mensch/spitzbergen-in-der-arktis-sorge-um-globalen-saatguttresor-a-1148789.html
74
https://www.spiegel.de//spiegel/print/d-151254608.html
https://www.spiegel.de//spiegel/print/d-151254653.html
https://www.spiegel.de//spiegel/print/d-151139741.html
https://www.spiegel.de//spiegel/spiegelwissen/d-150555035.html
https://www.spiegel.de/wissenschaft/natur/klimawandel-macht-antarktis-gruener-a-1148491.html
https://www.spiegel.de/wissenschaft/natur/klimawandel-forscher-erkl

https://www.spiegel.de/wissenschaft/natur/klimaschutzplan-2050-regierung-in-der-klimafalle-a-1119357.html
82
https://www.spiegel.de//spiegel/print/d-147472221.html
https://www.spiegel.de//spiegel/print/d-147350609.html
https://www.spiegel.de/politik/deutschland/klimaplan-barbara-hendricks-fordert-machtwort-von-angela-merkel-a-1119303.html
https://www.spiegel.de/wissenschaft/natur/spanien-und-sued-frankreich-klimawandel-koennte-suedeuropa-austrocknen-a-1118546.html
https://www.spiegel.de/wissenschaft/natur/500-sprudelnde-methanquellen-im-meeresboden-entdeckt-a-1117368.html
https://www.spiegel.de/wissenschaft/natur/klima-meteorit-verursacht-warmzeit-a-1116986.html
https://www.spiegel.de/wissenschaft/natur/treibhausgase-internationale-gemeinschaft-einigt-sich-in-ruanda-auf-reduzierung-von-fkw-a-1116761.html
https://www.spiegel.de/wissenschaft/natur/co2-emissionen-gefaehrliche-klimaschulden-a-1116522.html
https://www.spiegel.de/auto/aktuell/verbot-fuer-dieselautos-und-benzinautos-autoindus

https://www.spiegel.de/politik/ausland/usa-supreme-court-stoppt-obamas-klimaschutz-plaene-a-1076567.html
https://www.spiegel.de/wissenschaft/natur/treibhausgas-ausstoss-in-deutschland-2014-gesunken-a-1075486.html
https://www.spiegel.de/wissenschaft/mensch/us-klimaschutz-richter-bescheren-obama-zwischensieg-a-1073309.html
https://www.spiegel.de/wissenschaft/natur/republikaner-wollen-welt-klimavertrag-aushebeln-a-1073052.html
https://www.spiegel.de/wissenschaft/natur/klima-2015-war-waermstes-jahr-seit-beginn-der-messungen-a-1073035.html
https://www.spiegel.de/wissenschaft/natur/klima-die-naechste-eiszeit-faellt-aus-a-1071893.html
https://www.spiegel.de/wissenschaft/technik/co2-filter-ein-kescher-fuers-treibhausgas-a-1066890.html
91
https://www.spiegel.de//spiegel/print/d-140604332.html
https://www.spiegel.de//spiegel/print/d-140508737.html
https://www.spiegel.de//spiegel/print/d-140508736.html
https://www.spiegel.de/wissenschaft/natur/klimawandel-heizt-seen-auf-auch-in-deutschland-a-1068

98
https://www.spiegel.de//spiegel/print/d-139456048.html
https://www.spiegel.de/wissenschaft/mensch/klimaplaene-uno-hat-co2-ziele-der-staaten-zusammengerechnet-a-1060267.html
https://www.spiegel.de/wissenschaft/natur/fluechtlinge-klimawandel-und-wassermangel-verschaerfen-gefahr-a-1059195.html
https://www.spiegel.de/wissenschaft/natur/klimawandel-koennte-golfregion-bis-2100-unbewohnbar-machen-a-1059744.html
https://www.spiegel.de/wissenschaft/natur/uno-klimakonferenz-weltgemeinschaft-einigt-sich-auf-entwurf-fuer-klimavertrag-a-1059424.html
https://www.spiegel.de/wirtschaft/unternehmen/kakao-in-ghana-der-klimawandel-bedroht-die-ernte-a-1059187.html
https://www.spiegel.de/auto/aktuell/vw-abgasaffaere-eu-staaten-sehen-klimaziele-in-gefahr-a-1058552.html
https://www.spiegel.de/wissenschaft/natur/antarktis-eisschmelze-koennte-sich-bis-2100-versechsfachen-a-1057517.html
https://www.spiegel.de/wissenschaft/mensch/klimafonds-weltbank-gibt-pro-jahr-16-milliarden-dollar-a-1057175.html
https://ww